In [3]:
import pandas as pd
import dvc.api
import sys
sys.path.insert(0,'../scripts/')

In [4]:
def get_data(path,repo,version):
    data_url = dvc.api.get_url(
    path=path,
    repo=repo,
    rev=version
    )
    df = pd.read_csv(data_url)
    return df

In [ ]:
driver_location_data = get_data('data/driver_locations_during_request.csv','C:/Users/User/Desktop/Causal-Inference','driver_locations_during_request_v1')
nb_data = get_data('data/nb.csv','C:/Users/User/Desktop/Causal-Inference','nb_v1')